In [1]:
from pydataset import data
import pandas as pd
import numpy as np

import datetime

In [ ]:
# 1. Load the mpg dataset. Read the documentation for it, and use the data to answer 
# these questions:

data('mpg', show_doc=True) # view the documentation for the dataset
# with show_doc=True won't assign data to variable

mpg = data('mpg') # load the dataset and store it in a variable



In [ ]:
# On average, which manufacturer has the best miles per gallon?

# add column of average mpg
mpg['avg_mpg'] = (mpg.cty + mpg.hwy) / 2

# average mpg per manufacturer best on top
mpg.groupby('manufacturer').avg_mpg.mean().sort_values(ascending=False)


In [ ]:
# How many different manufacturers are there?

# use .unique() to get list of unique values, len() will give total count
# https://cmdlinetips.com/2018/01/how-to-get-unique-values-from-a-column-in-pandas-data-frame/
len(mpg.manufacturer.unique())

In [ ]:
# How many different models are there?
# use .unique() to get list of unique values, len() will give total count
len(mpg.model.unique())

In [ ]:
# Do automatic or manual cars have better miles per gallon?
mpg.head()
bools = mpg.trans.str.contains('man')
print("Cars with a manual transmission = True, Automatic Transmission = False")
print("Cars with a manual transmission have a higher average mpg")
mpg.groupby(bools).avg_mpg.mean().sort_values(ascending=False)

In [ ]:
# 2. Joining and Merging
# Copy the users and roles dataframes from the examples above. 

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})


# What do you think a right join would look like? 

# a right join would show all roles (even those without users), but not all users
pd.merge(users, roles, left_on='role_id', right_on='id', how='right')


In [ ]:
#  An outer join?

# an outer join should show all users and all roles inserting null values where needed
pd.merge(users, roles, left_on='role_id', right_on='id', how='outer')

In [ ]:
# What happens if you drop the foreign keys from the dataframes and try to merge them?

# it puts the roles as additional names below the original users
pd.merge(users, roles, how='outer')

In [2]:
# 3. Getting data from SQL databases
# 3.a. Create a function named get_db_url. It should accept a username, hostname, 
# password, and database name and return a url formatted like in the examples 
# in this lesson.

# from env import host, user, password
# url = f'mysql+pymysql://{user}:{password}@{host}/employees'
# pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)


# for getting from Codeup SQL databases
def get_db_url(database):
    from env import host, user, password
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url

# note: this would require typing user and password into jupyter notebook
# which would defeat the purpose of annonimity!
# def general_get_db_url(user, password, host, database):
#     url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
#     return url


In [ ]:
# Ryan's challenge = show all available databases
from env import host, user, password
url = f'mysql+pymysql://{user}:{password}@{host}'
query = '''show databases'''
pd.read_sql(query, url)

In [ ]:
# 3.b. Use your function to obtain a connection to the employees database.
# (and run a query)

query = """select * from orders limit 100"""
pd.read_sql(query, get_db_url('chipotle'))


In [ ]:
# 3. Once you have successfully run a query:
# 3.c.i. Intentionally make a typo in the database url. What kind of error message do you see?

# misspell database name
#query = """select * from orders limit 100"""
#pd.read_sql(query, get_db_url('chipotel'))

# spelling error in url, supposed to be pymysql
url = f'mysql+pysql://{user}:{password}@{host}'
query = '''show databases'''
pd.read_sql(query, url)


In [ ]:
# 3.c.ii. Intentionally make an error in your SQL query. What does the error message look like?

# misspell query table name
query = """select * from oders limit 100"""
pd.read_sql(query, get_db_url('chipotle'))


In [3]:
# 3.d. Read the employees and titles tables into two separate dataframes

# write SQL queries for tables
query_employees = '''select * from employees'''
query_titles = '''select * from titles'''
# url function
url = get_db_url('employees')
# assign tables to df variables
employeesdf = pd.read_sql(query_employees, url)
titlesdf = pd.read_sql(query_titles, url)


In [ ]:
# 3.e. Visualize the number of employees with each title.

# 3.e.i. current data
# get a sense of what titlesdf looks like
titlesdf.head()
# set today's date variable and create bools to filter to_date greater than today
today = pd.to_datetime('today')
bools = titlesdf.to_date > today 
# use the groupby to count the number of employees with that title currently
titlesdf[bools].groupby('title').count()

In [ ]:
# 3.e.ii. Visualize the number of employees with each title -- for all titles historical
# this will count the historical number of employees with that title
titlesdf.groupby('title').count()

In [4]:
# 3.f. Join the employees and titles dataframes together.

# get a sense of the columns in the df to join
print(employeesdf.head())
print(titlesdf.head())

# merge dfs on emp_no
emp_with_titles = pd.merge(employeesdf, titlesdf, left_on='emp_no', right_on='emp_no', how='outer')
# show combined head
emp_with_titles.head()

   emp_no  birth_date first_name last_name gender   hire_date
0   10001  1953-09-02     Georgi   Facello      M  1986-06-26
1   10002  1964-06-02    Bezalel    Simmel      F  1985-11-21
2   10003  1959-12-03      Parto   Bamford      M  1986-08-28
3   10004  1954-05-01  Chirstian   Koblick      M  1986-12-01
4   10005  1955-01-21    Kyoichi  Maliniak      M  1989-09-12
   emp_no            title   from_date     to_date
0   10001  Senior Engineer  1986-06-26  9999-01-01
1   10002            Staff  1996-08-03  9999-01-01
2   10003  Senior Engineer  1995-12-03  9999-01-01
3   10004         Engineer  1986-12-01  1995-12-01
4   10004  Senior Engineer  1995-12-01  9999-01-01


,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


In [ ]:
type(emp_with_titles.to_date[1])

In [10]:
emp_with_titles['to_date'] - emp_with_titles['from_date']

OverflowError: Python int too large to convert to C long

In [11]:
# 3.g. Visualize how frequently employees change titles.

# find type of data in to_date and from_date columns
type(emp_with_titles.to_date)

# exobrain http://chris35wills.github.io/time_elapsed_pandas/
# columns are objects, pandas timedelta works on objects
# per exobrain https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timedelta.html

# replace future to_date to today for calculations
#df['column name'] = df['column name'].replace(['old value'],'new value')
# today = pd.to_datetime('today')
# print(today)
# to = pd.to_datetime(emp_with_titles.to_date, yearfirst=True) 
today = pd.to_datetime('today')
bools = titlesdf.to_date < today 
emp_with_titles['calc_date'] = emp_with_titles.to_date[bools]

# df_test['Difference'] = df_test['First_Date'].sub(df_test['Second Date'], axis=0)
# df_test.head()

emp_with_titles['to_date'] - emp_with_titles['from_date']


# calculate time in position for all (include historic) and add as column
# exobrain http://www.datasciencemadesimple.com/difference-two-dates-days-weeks-months-years-pandas-python-2/
#emp_with_titles['time_in_position'] = emp_with_titles['to_date'] - emp_with_titles['from_date']
#emp_with_titles['time_in_position']=df['diff_years']/np.timedelta64(1,'Y')


OverflowError: Python int too large to convert to C long

In [ ]:
# For each title, find the hire date of the employee that was hired most recently 
# with that title.

In [ ]:
# Write the code necessary to create a cross tabulation of the number of titles
# by department. (Hint: this will involve a combination of SQL and python/pandas code)

In [ ]:
# 4. Use your get_db_url function to help you explore the data from the chipotle database. 
# Use the data to answer the following questions:






In [ ]:
# What is the total price for each order?

In [ ]:
# What are the most popular 3 items?

In [ ]:
# Which item has produced the most revenue?